# Distributed Swarm Learning Model

## Created DSL Model


*  Defined Conv2D Architecture
*  Defined PSO Hyperparamers -- w, c1 and c2 respectively
*  Number of Agents/Clients Used = 5
*  Initialized velocities
*  Splitted dataset among agents
*  Number of Epochs = 5


**Imports**:


1.   Tensorflow
2.   Keras
3.   Numpy
4.   Sklearn

In [ ]:
# Authors: Sai Prasad, Purnima Sai Koumudi
# Model saved time: 04/25

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

**Downloading the dataset (CIFAR 10)**

Data Preprossing

In [ ]:
# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train.astype('float32') / 255.0, x_test.astype('float32') / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

**CNN Model Architecture**

Optimizers used: Adam

Loss: Categorical Crossentropy

Activation: RELU


In [ ]:
# Define the CNN model architecture
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

**Initialized PSO Hyperparameters**

In [ ]:
global_model = create_cnn_model()

# PSO parameters initialization
w = 0.9  # Inertia weight
c1, c2 = 0.5, 0.5  # Cognitive and social coefficients

**Defined Number of Agents**

In [ ]:
# Initialize agents
num_agents = 5
agents = [create_cnn_model() for _ in range(num_agents)]
personal_best_weights = [agent.get_weights() for agent in agents]
personal_best_loss = [float('inf')] * num_agents

**Initialized Velocities**

In [ ]:
# Initialize velocities
def initialize_velocity(models):
    return [[np.zeros_like(weights) for weights in model.get_weights()] for model in models]

velocity = initialize_velocity(agents)

**Splitting the dataset among agents**

In [ ]:
# Split dataset among agents
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train, y_train = x_train[indices], y_train[indices]
splits = np.array_split(indices, num_agents)

**Training the model defined**

In [ ]:
# Training loop
epochs = 25
last_global_loss = float('inf')

for epoch in range(epochs):
    for i, agent in enumerate(agents):
        idx = splits[i]
        x_split, y_split = x_train[idx], y_train[idx]
        x_agent_train, x_agent_val, y_agent_train, y_agent_val = train_test_split(x_split, y_split, test_size=0.1)

        history = agent.fit(x_agent_train, y_agent_train, validation_data=(x_agent_val, y_agent_val), epochs=1, batch_size=32, verbose=0)
        val_loss = history.history['val_loss'][0]

        if val_loss < personal_best_loss[i]:
            personal_best_loss[i] = val_loss
            personal_best_weights[i] = agent.get_weights()

        for weight_idx, (current_weights, v) in enumerate(zip(agent.get_weights(), velocity[i])):
            r1, r2 = np.random.random(), np.random.random()
            v = w * v + c1 * r1 * (personal_best_weights[i][weight_idx] - current_weights) + c2 * r2 * (global_model.get_weights()[weight_idx] - current_weights)
            current_weights += v
            velocity[i][weight_idx] = v

        agent.set_weights([w + v for w, v in zip(agent.get_weights(), velocity[i])])

**Updation of Global Best Model**

In [ ]:
# Update global model to the best performing agent's weights
    best_agent_index = np.argmin(personal_best_loss)
    global_model.set_weights(personal_best_weights[best_agent_index])
    global_loss, global_accuracy = global_model.evaluate(x_test, y_test, verbose=0)

    if global_loss < last_global_loss:
        last_global_loss = global_loss
    else:
        w *= 0.95  # Decrease inertia to reduce oscillations
        c1 *= 1.05  # Increase cognitive exploration
        c2 *= 1.05  # Increase social learning

    print(f"Global Model - Epoch {epoch+1}: Test Loss: {global_loss:.4f}, Test Accuracy: {global_accuracy:.4f}")


170498071/170498071 [==============================] - 3s 0us/step
Global Model - Epoch 1: Test Loss: 1.9542, Test Accuracy: 0.2202
Global Model - Epoch 2: Test Loss: 1.8415, Test Accuracy: 0.2650
Global Model - Epoch 3: Test Loss: 1.8009, Test Accuracy: 0.3131
Global Model - Epoch 4: Test Loss: 1.7216, Test Accuracy: 0.3437
Global Model - Epoch 5: Test Loss: 1.6270, Test Accuracy: 0.3822
Global Model - Epoch 6: Test Loss: 1.5173, Test Accuracy: 0.4452
Global Model - Epoch 7: Test Loss: 1.4405, Test Accuracy: 0.4675
Global Model - Epoch 8: Test Loss: 1.4071, Test Accuracy: 0.4785
Global Model - Epoch 9: Test Loss: 1.3661, Test Accuracy: 0.5047
Global Model - Epoch 10: Test Loss: 1.2978, Test Accuracy: 0.5207
Global Model - Epoch 11: Test Loss: 1.2978, Test Accuracy: 0.5207
Global Model - Epoch 12: Test Loss: 1.2978, Test Accuracy: 0.5207
Global Model - Epoch 13: Test Loss: 1.2798, Test Accuracy: 0.5370
Global Model - Epoch 14: Test Loss: 1.2798, Test Accuracy: 0.5370
Global Model - Epo

**Model Saving Checkpoint**

In [ ]:
global_model.save("trained_model_3.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
